Importing libraies and reading data csv file into a Pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df_tor = pd.read_csv(body)
df_tor.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Rename columns

In [4]:
df_tor.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove all rows that have a Borough Not Assigned

In [5]:
df_tor['Borough'].replace('Not assigned',np.nan, inplace=True)
df_tor.dropna(subset=['Borough'], axis=0, inplace=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combine rows with the same Postal Code <br/>
Replace Not Assigned Neighborood values with their corresponding Borough value <br/>
Randomize dataframe rows

In [6]:
df_tor=df_tor.groupby(['PostalCode','Borough'], as_index =False).agg({'Neighborhood': ', '.join})
df_tor['Neighborhood'].replace('Not assigned',df_tor['Borough'], inplace=True)
df_tor = df_tor.sample(frac=1).reset_index(drop=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
1,M1G,Scarborough,Woburn
2,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
3,M1J,Scarborough,Scarborough Village
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."


In [7]:
df_tor.shape

(103, 3)

The datframe has 103 rows or records and 3 columns or features

In [8]:
body = client_d6b108fb8f9a44b6b683b7059b7338a1.get_object(Bucket='myfirstproject-donotdelete-pr-qxcxkzyfq9az74',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_coords = pd.read_csv(body)
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join the coordinates dataframe with the original dataframe on the matching Postal Code of both dataframes

In [9]:
df_tor=df_tor.join(df_coords.set_index('Postal Code'), on='PostalCode')
df_tor.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
1,M1G,Scarborough,Woburn,43.770992,-79.216917
2,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
5,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M9L,North York,Humber Summit,43.756303,-79.565963
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3N,North York,Downsview Northwest,43.761631,-79.520999


In [10]:
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

Narrow down the datframe to only records whose boroughs contain the word Toronto

In [11]:
tor_data = df_tor[df_tor['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_data.head

<bound method NDFrame.head of    PostalCode           Borough  \
0         M5T  Downtown Toronto   
1         M6R      West Toronto   
2         M4T   Central Toronto   
3         M7A  Downtown Toronto   
4         M5J  Downtown Toronto   
5         M5E  Downtown Toronto   
6         M5R   Central Toronto   
7         M7Y      East Toronto   
8         M5L  Downtown Toronto   
9         M4M      East Toronto   
10        M5P   Central Toronto   
11        M4E      East Toronto   
12        M4K      East Toronto   
13        M4V   Central Toronto   
14        M6H      West Toronto   
15        M5H  Downtown Toronto   
16        M4N   Central Toronto   
17        M4X  Downtown Toronto   
18        M5G  Downtown Toronto   
19        M4L      East Toronto   
20        M5S  Downtown Toronto   
21        M4S   Central Toronto   
22        M5X  Downtown Toronto   
23        M5V  Downtown Toronto   
24        M6G  Downtown Toronto   
25        M4P   Central Toronto   
26        M5K  Downtown T

Defining a user_agent for an instance of the geocoder centered at the City of Toronto

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Created a map of Toronto using corridnates <br/>
Created markers for each group of neighborhoods that have the same Postal Code <br/>
Labeled each group with list of neighbrhoods, borough, and Postal Code

In [13]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, postalcode, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['PostalCode'], tor_data['Borough'], tor_data['Neighborhood']):
    label = '{}, {} {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

You can observe there are more clusters of neighborhoods the closer we get to the central coordinate of downtown Toronto

In [14]:
import requests
from pandas.io.json import json_normalize

In [15]:
CLIENT_ID = 'FJDKZYMXDRHIO4W302BOSCD0YVJ33WAC1CBJEND4O01AU5AD'
CLIENT_SECRET = '1FYAET1WSZDR24NYCTRL5ZSXAKCVTIESTI4OPAOPW0XWKRNA'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FJDKZYMXDRHIO4W302BOSCD0YVJ33WAC1CBJEND4O01AU5AD
CLIENT_SECRET:1FYAET1WSZDR24NYCTRL5ZSXAKCVTIESTI4OPAOPW0XWKRNA


Make a request from the Foursquare API to return Venue name, lagitude, longitude, and category in each neighborhood of Tornto

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
tor_venues = getNearbyVenues(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Chinatown, Grange Park, Kensington Market
Parkdale, Roncesvalles
Moore Park, Summerhill East
Queen's Park
Harbourfront East, Toronto Islands, Union Station
Berczy Park
The Annex, North Midtown, Yorkville
Business Reply Mail Processing Centre 969 Eastern
Commerce Court, Victoria Hotel
Studio District
Forest Hill North, Forest Hill West
The Beaches
The Danforth West, Riverdale
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Dovercourt Village, Dufferin
Adelaide, King, Richmond
Lawrence Park
Cabbagetown, St. James Town
Central Bay Street
The Beaches West, India Bazaar
Harbord, University of Toronto
Davisville
First Canadian Place, Underground city
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Christie
Davisville North
Design Exchange, Toronto Dominion Centre
St. James Town
Stn A PO Boxes 25 The Esplanade
Rosedale
Roselawn
North Toronto West
Harbourfront
Ryerson, Garden District
Church and Wellesley
Little Portu

In [18]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,Kid Icarus,43.653933,-79.401719,Arts & Crafts Store
1,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,Mexican Restaurant
2,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,Essence of Life Organics,43.654111,-79.400431,Organic Grocery
3,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,Jimmy's Coffee,43.654493,-79.401311,Café
4,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,Blackbird Baking Co,43.654764,-79.400566,Bakery


In [19]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",18,18,18,18,18,18
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,18,18,18,18,18,18


One hot encoding for all Venue categorys for evey venue pulled <br/>
Include the neigborhood of each venue in the dataframe

In [20]:
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
tor_onehot['Neighborhood'] = tor_venues['Neighborhood']

Group all venues by neighbohood and get mean frequency of each venue category in each neighborhood

In [21]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


List out the top five venue categories in each neighborhood

In [22]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                venue  freq
0  Seafood Restaurant  0.07
1                Café  0.07
2               Hotel  0.07
3    Asian Restaurant  0.07
4    Sushi Restaurant  0.07


----Berczy Park----
                venue  freq
0      Farmers Market  0.07
1                Café  0.07
2         Coffee Shop  0.07
3            Beer Bar  0.07
4  Seafood Restaurant  0.07


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3   Grocery Store  0.04
4    Intersection  0.04


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2           Park  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Termin

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new data fram that iincludes each neighbood and their most common venue category from first to tenth

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Seafood Restaurant,Hotel,Asian Restaurant,Sushi Restaurant,Greek Restaurant,Food Court,Monument / Landmark,Plaza,Smoke Shop
1,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Café,Farmers Market,Seafood Restaurant,Beer Bar,Museum,Creperie,Park
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Climbing Gym,Pet Store,Convenience Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Skate Park,Burrito Place,Comic Shop,Spa,Farmers Market,Fast Food Restaurant,Butcher,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique


Use the K  menas clustering algorithm to cluster groups of neighbrhoods based on the similarity of most common venue categories

In [45]:
# set number of clusters
kclusters = 4

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

Add clustering labels and merge the original dataset with the dataset containing most common venue categories

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [46]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The map shows most neighboroods directly in and around downtown Toronto fall in one cluster <br/>
There are a few neighborhoods north of downtown Tornto that fall in different clusters